In [ ]:
# required imports
from firebolt.client import FireboltResourceClient

# use debug function for pretty printing
from devtools import debug

# configure logging
import logging

logging.basicConfig(
    format="{asctime} - {name} - {levelname} - {message}", style="{", level="INFO"
)

# show every web request
logging.getLogger("firebolt.http_client").setLevel("DEBUG")

Set an engine_name and database_name you want to operate on before proceeding

In [ ]:
engine_name = "eg_temp"
database_name = "eg_temp"
assert engine_name
assert database_name

### Inspect Configuration
Run the following cell to make sure you can connect to Firebolt, and that your environment variables or .env file is being read properly.

In [ ]:
from firebolt.common import Settings

settings = Settings(
    user="",
    password="",
    server="",
    default_region="",  # us-east-1
)

with init_firebolt_client() as fc:
    debug(fc.base_url)

In [ ]:
settings = None

### Run a query

In [ ]:
from firebolt.model.engine import Engine

sql = "SELECT 1;"

with FireboltResourceClient():
    engine = Engine.get_by_name(engine_name=engine_name)
    response = engine.run_query(sql=sql)

debug(response)

### Get existing Engine by name, start, stop

In [ ]:
from firebolt.service.engine_service import EngineService

with FireboltResourceClient(settings) as fc:
    engine_service = EngineService(fc)

    engine = engine_service.get_engine_by_name(engine_name=engine_name)
    debug(engine)

    started_engine = engine_service.start_engine(engine=engine)
    debug(started_engine)

    stopped_engine = engine_service.stop_engine(engine=engine)
    debug(stopped_engine)

### Create default general purpose (ingest) engine
Note: there is also `Engine.create_analytics(...)` which works the same way as `create_general_purpose(...)`.

In [ ]:
from firebolt.service.engine_service import EngineService

with FireboltResourceClient(settings) as fc:
    engine_service = EngineService(fc)
    engine = engine_service.create_general_purpose_engine(
        name=engine_name,
        compute_instance_type_name="m5d.4xlarge",
    )
    debug(engine)

### Copy an Engine

In [ ]:
from firebolt.service.engine_service import EngineService

with FireboltResourceClient(settings) as fc:
    engine = EngineService(fc).get_engine_by_name(engine_name=engine_name)
    engine.name = f"{engine_name}_copy"
    debug(engine)
    new_engine = EngineService.create_engine(engine=engine)
    debug(new_engine)

### Get engines for a database

In [ ]:
from firebolt.service.database_service import DatabaseService
from firebolt.service.binding_service import BindingService


with FireboltResourceClient(settings) as fc:
    database = DatabaseService(fc).get_by_name(database_name=database_name)
    engines = BindingService(fc).get_engines_bound_to_database(database=database)

debug(engines)

### Create engine, database, and bind them

In [ ]:
from firebolt.service.database_service import DatabaseService
from firebolt.service.engine_service import EngineService

with FireboltResourceClient(settings) as fc:
    engine_service = EngineService(fc)
    database = DatabaseService(fc).get_by_name(database_name=database_name)
    engine = engine_service.create_general_purpose_engine(
        name=engine_name, compute_instance_type_name="m5d.4xlarge"
    )
    binding = engine_service.bind_engine_to_database(
        engine=engine, database=database, is_default_engine=True
    )

    debug(binding)

### Get default provider, region, and an instance_type
An example of these might be:
 * provider: AWS
 * region: us-east-1
 * instance_type: i3.4xlarge

In [ ]:
with FireboltResourceClient() as fc:
    debug(fc.provider_service.default_provider)
    debug(fc.region_service.default_region)
    debug(fc.instance_type_service.get_by_name(instance_type_name="i3.4xlarge"))